# Tutorial 6: Basics of netCDF file manipulation

In previous tutorials, we've worked with discrete data such as hourly air quality measurements. In this tutorial, we'll move beyond discrete data into continuous or gridded data. Specifically, we'll use satellite data, but this method can also be used with model data, reanalysis data, or any dataset that is continuous or gridded over the area of interest.   

By the end of this tutorial, you will be able to:
* open netCDF files with xarray
* distinguish between code for DataArrays and Datasets
* identify the properties of an xarray DataArray and Dataset
* create a plot using continuous or gridded data
* manipulate data using array properties

### How to deal with 2D or 3D data in Python

So far, we've worked with lists and tables, which are examples of 1D (lists) and simple 2D (tables) data. But what if we have more complex 2D data, or even 3D data? pandas can no longer handle this type of information. Instead, we use other packages like *xarray*.

In [ ]:
# for data manipulation and opening files
import xarray as xr
import pandas as pd

# for plotting
import matplotlib.pyplot as plt
from cartopy import crs, feature

### Downloading data

Files of large gridded datasets can be too large to easily share. Instead, you will need to download the data yourself. In later tutorials, we will show ways to download data with Python code and ways to work with cloud-based data without downloading. 
1. Go to https://sage.nelson.wisc.edu/data-and-models/datasets/satellite-data-for-air-quality/. 
2. Choose a month and year to download. It doesn't matter which month and year, but be sure to remember your choice. 
3. Download the tar file to the same directory (folder) as this tutorial. 
4. Double-click on the downloaded file to untar and unzip it. You should now have a folder called DOMINO_NO2_yourchosendate. Inside this folder, there should be netCDF files (.nc) for every day within that month. 
5. Pick any day to work with for the rest of this tutorial. You can come back later and change your choice. 

The data that you have just downloaded is NO$_{2}$ measurements from the OMI satellite. The level 2 satellite data were re-gridded over the United States using the Wisconsin Horizontal Interpolation Program for Satellites (WHIPS) code. 

Satellite data is very useful for studying regional or global trends in air quality because they are able to make measurements over large areas. However, most satellites have a limit to their resolution (how small of an area they can resolve, or see), so trying to zoom in on a very small spot is difficult. For this reason, we will be using measurements over the whole US, rather than zooming in on a small region around a city. 

### Opening files with xarray

In [ ]:
# for the year, month, and day you selected, fill in the appropriate information
# change the path to your downloaded file if necessary

year = '2020'
month_name = 'Jan'
month_num = '01'
day = '01'
path = 'DOMINO_NO2_'+month_name+year
file = '/WHIPS-NO2-12kmCONUS_'+month_num+'-'+day+'-'+year+'.nc'

<font color=red>**Note:**</font> Some of the files, especially the earlier files, have different naming formats. If you get an error message in the code cell below, double-check that the name of the file is in the same format listed above.

In [ ]:
# use xarray to open the file you chose
data = xr.open_dataset(path+file)

xarray can be described as a multi-dimensional version of pandas. Like pandas, the data can be indexed and sorted based on location or name. However, xarray can do these things with data of three dimensions or more. Let's explore some of the ways xarray organizes the data.

In [ ]:
# what does this data look like?
data

Ok, there's a lot going on here. Let's break these different things down a bit.    

**Array Type:** The very first thing that is shown in the above print out is 'xarray.Dataset'. There are two types of arrays that xarray can handle: DataArrays and Datasets. A Dataset is a collection of DataArrays. Imagine that a DataArray is a rubix cube. It is a 3D (or 2D or 4D or however shaped) array of data. A Dataset would therefore be a collection of rubix cubes. Imagine you ordered a shipment of 30 rubix cubes. Each individual rubix cube is a DataArray, and the box that all the cubes were shipped in is the Dataset. We can select one DataArray using its variable name. There are some data selection methods that only work on DataArrays, which is why it is important to distinguish between DataArrays and Datasets.

**Dimensions:** The dimensions are the directions that the file has, or the names of the axes. There are three dimensions listed in this file: rows, columns, and layers. Imagine your 3D rubix cube again. We can label the directions of the cube with row, column, and layer. You could think of these as the x, y, and z directions. Since this is satellite data, we can also think of these as latitudes, longitudes, and heights. There are 246 rows, which means there are 246 lines of latitude in the grid, or 246 rubix cube squares along one edge. There are 396 columns, or 396 lines of longitude, or 396 rubix cube squares along the second edge. Finally, there are 34 layers, or 34 heights, or 34 rubix cube squares going up to make the grid three dimensional.   

**Coordinates:** Coordinates are labels for each step of a dimension. This particular file has no coords, so lets imagine some here. There are 34 layers, or heights, in the third dimension in this file. We could label each layer with what might be the corresponding region of the atmosphere. For example, the first 3 layers might be ["surface","planetary boundary","free atmosphere"]. In the code cell below, we separate out the first 3 layers and then label them with these coords. 

In [ ]:
layers = ["surface","planetary boundary","free atmosphere"]
data_subset = data[dict(layer=slice(None, 3))].assign_coords(layer=layers) # we'll explain this later!
data_subset

**Data Variables:** These are the variables of interest. This is the actual data in your file. For example, in this file there is "time", "avKern", "tropVCD", "CloudFraction", and "ValidPixelCount" that might be of interest to us. Many of the data variables have unusual names, which can make understanding the data difficult. xarray is great because it allows you to have metadata for each data variable, which might help to explain the data a little more. Click on the picture of a piece of paper next to each variable name. Some of the data variables have useful metadata under this piece of paper. Using this information, can you guess what some of the variables mean?   
Next, click on the picture of 3 stacked circles beside a data variable. What should appear is an array of the data. These are the actual values of this data variable.  

**Attributes:** Finally, xarray has more metadata stored in the Attributes. Some of this information can be helpful, such as the time_comparison = UTC or the projection = lcc2par. Knowing your data is in UTC time is super important, and knowing the map projection is important for properly formatting the data for visualization. Based on the previous cartopy tutorial, can you guess what projection lcc2par is?

If you are interested in knowing just the attributes or just the coordinates or just the dims of an xarray DataArray, you can call this information specifically.

In [ ]:
# here, we call the dimensions using the key word "dims"
data_subset.dims

In [ ]:
# here, we call the coordinates using the key word "coords"
data_subset.coords

In [ ]:
# here, we use the key word "attrs". Can you guess what this key word calls?
data_subset.attrs

In [ ]:
# here, we call the data variables using the key word "var"
data_subset.var

### Selecting and indexing data

Remember earlier when we said that DataArrays and Datasets are different? Indexing and selecting data is one of the places that distinguish DataArrays and Datasets. Right now, we have a Dataset. This is a collection of DataArrays. We can select one of these arrays using the variable name.

In [ ]:
# get a DataArray from our Dataset
# like using column names in pandas, we can select specific DataArrays using the array variable name
var = data_subset['avKern']

In [ ]:
# look at the data
var

**Knowledge Check:** How is this print out different from the print out of data_subset? Why is it different? Take a moment here to really think through the differences between var and data_subset and what they mean. Some information you might want to think about for the next few code cells:
1. What type array is this?
2. How many dimensions are there?

Like pandas, there are several ways to select data in xarray. We can select the dimension of the data using positions or using names. Using positions, this is like saying "I want the data in axis 2". Using names, this is like saying "I want the data in the layers". For either of these methods, you can use an integer or a label to select the index. We'll try a few examples below.

In [ ]:
# positional indexing of dimensions using integers
# translation: use a number to select the axis
var[:,:,0]

In the example above, we used square brackets ```[]``` to say "Take the dims of this DataArray." We then used colons ```:``` to say "Take all points in this dimension" and an integer ``0`` to say "take the 0th point in this dimension." The code ```var[:,:,0]``` therefore is saying "Take all values in the rows and columns, but only in the 0th layer." You can see how this was done by looking at the new dims of the data.

In [ ]:
# positional indexing of dimensions using labels
# translation: use a coordinate label to select the axis
var.loc[:,:,'surface']

In this example, we used the coordinate labels to select only one layer of the data. We now have all the rows and columns of data for the surface layer only. This is the same result as the first indexing example, but using a label instead of an integer.

In [ ]:
# dimension name indexing using integers
var.isel(layer=0)

In this example, we used the `isel` (integer select) command to select only the 0th layer of the DataArray. This is the same result as the first two examples.

In [ ]:
# another way to index using dimension names and integers
var[dict(layer=0)]

In [ ]:
# dimension name indexing using labels (try both methods by commenting/uncommenting the lines)
var.sel(layer='surface')
# var.loc[dict(layer='surface')]

Here we used the dimension name again (layer) to specify which dimension we wanted, but we used a coordinate label to select the data of interest. This result is the same as the other methods we used. 

You should now be able to see how xarray, like pandas, has multiple methods to select and index data. This is only true for DataArrays. Datasets, because they are more complex, cannot be indexed using positions. Dimension names must be used. 

Remember this line of code `data_subset = data[dict(layer=slice(None, 3))].assign_coords(layer=layers)`? Here, we indexed the data and then assigned coordinate labels to the subset of layers we selected. The original Dataset that we loaded needed to be indexed using the dimension name method. We could not have used positional indexing. If we try using positional indexing, what happens?

In [ ]:
data_subset = data[:,:,0:3].assign_coords(layer=layers)

When we try to select a subset of data using positional indexing on a Dataset, we get a <font color=red>TypeError</font>: unhashable type: 'slice' message. This message is telling us that the Dataset can not be divided up into slices using `:` positional indexing. 

Ok, but what is a slice?   
A slice is what it sounds like: it is a piece of the data that we are cutting out from the whole. `slice(None, 3)` says "Cut a piece of this data, starting at the 0th position (None) and ending at the 3rd position (3)." We used the dimension name "layer" to tell xarray which dimension to index on. So `layer=slice(None,3)` is saying "Cut a piece of the layer data, starting at the 0th position and ending at the 3rd position." This is how we cut out the first 3 layers of the data.   
In the cell below, we print out the dimensions of the original data and of some sliced data. Do the dimensions printed match your expectations?

In [ ]:
print(data.dims)
print(data[dict(row=slice(None, 30))].dims)
print(data[dict(col=slice(25,100))].dims)

**Knowledge Check:** If you see code that has square brackets `data[:]`, is that code selecting data using positional indexing or dimension name indexing? Which type of array can use this indexing method?

### Data visualization

Now that we know how to select the data of interest, let's work on visualizing our data. xarray has built-in plotting abilities, but these abilities are shortcuts of the same Matplotlib commands we learned in tutorials 3 and 4. To be more explicit in how we are visualizing and manipulating the data, we will continue to use the Matplotlib commands.

In [ ]:
# select a variable of interest for plotting
NO2 = data['tropVCD']

In [ ]:
# look at the data
NO2

This data is the concentration of NO$_{2}$ molecules in the troposphere. The dimensions of the data are rows and columns, there is no layer dimension. Therefore, we could plot this data of rows (latitudes) and columns (longitudes) using a grid that is 246 x 396, or we could make line plots of average concentrations over one of these dimensions. 

What if we wanted to know the average concentration along a latitude or longitude? Perhaps we have the hypothesis that NO$_{2}$ concentration is higher at lower latitudes closer to the equator. How would we test this hypothesis? We could take the mean of the data along the axis of interest.

The NO$_{2}$ data is formatted in a 2D array, and we want to take the average along one of these dimensions. If we want to know the average at each latitude, then we must take the mean along the column dimension. If this is hard to visualize, think back to a rubix cube. We want to know the average at each point along the y-axis (the rows). In order to do this, we must smush the x-axis, so take the average of the columns. 

Once again, we can using different indexing methods in xarray to accomplish the same task. Now let's plot the averages, this time using xarray plotting.

In [ ]:
# find the mean of the rows (latitudes) using integers
lat_means = NO2.mean(axis=1)
lat_means.dims

In [ ]:
# find the mean of the rows (latitudes) using dimension names
lat_means2 = NO2.mean('col')
lat_means2.dims

In [ ]:
fig = plt.figure(figsize=(10,6))
lat_means.plot()
plt.show()

Conveniently, xarray plotting can use the dimensions and the data variables to correctly label the axes. However, these labels aren't the best for professional presentations, so you can still use Matplotlib commands to clean up the figure.

In [ ]:
fig = plt.figure(figsize=(10,6))
lat_means.plot(c='k',label='Jan 1, 2020') # you can change the label data for your data
plt.xlabel('Row',fontsize=12)
plt.ylabel('NO$_{2}$ (molecules/cm$^{2}$)',fontsize=12)
plt.legend(fontsize=12)
plt.show()

### Merging data

So far, we've been manipulating data from one Dataset of data collected on one date. What if we want to look at data from more than one date? We downloaded the data for an entire month, can we merge two or more dates together?

In [ ]:
# first, open another dataset
year = '2020'
month_name = 'Jan'
month_num = '01'
day = '02'
path = 'DOMINO_NO2_'+month_name+year
file = '/WHIPS-NO2-12kmCONUS_'+month_num+'-'+day+'-'+year+'.nc'

In [ ]:
# use xarray to open the file you chose
data2 = xr.open_dataset(path+file)

In [ ]:
# look at the data
data2

At first glance, this Dataset looks pretty similar to the first Dataset. The dimensions, coordinates, and data variables are the same. Let's combine this Dataset with the first Dataset, but create a new "date" dimension.

In [ ]:
merge = xr.concat([data,data2],dim='date')
merge

We now have one larger Dataset that includes data and data2. These two original Datasets are differentiated by their position in the date dimension. We can label the dates by assigning coordinates.

In [ ]:
# the labels of the coords can be pandas datetimes
# change dates for your data, and comment out the methods that don't work for you

# if you have consecutive days, you can use pd.date_range
dates = pd.date_range('01-01-2020','01-02-2020') 

# if you have non-consecutive days, but if they repeat in a sequence, you can still use pd.date_range
# just specify the frequency of the day intervals
dates = pd.date_range('01-01-2020','01-02-2020',freq='1D') # change the number before D (day)

# if you have completely random days, you can make a list of dates then use pd.to_datetime
date_list = ['01-01-2020','01-02-2020']
dates = pd.to_datetime(date_list)

In [ ]:
dates

In [ ]:
merge = merge.assign_coords(date=dates)
merge.coords

There are now 4 total dimensions in the Dataset, with "date" being the first dimension. This new Dataset is the combination of the two individual Datasets. In the following exercises, you will practice indexing this new Dataset and merging another file. 

### Exercises

**1.** Select a subset of data from the merge Dataset that is   
a) all dates, the first 200 rows, all columns, and all layers

b) all dates, all rows, columns 100 to 200, and the last layer

c) all dates, the 3rd row, columns 20 to 50, and the first layer which should be labeled "surface"

**2.** Select the avKern data from the merge Dataset. Find the following means of this DataArray.   
a) Mean of the rows on each day and at each layer (246 x 2 x 34 data points)

b) Mean of the layers on each day (2 x 34 data points)

c) Mean of each day (2 data points)

**3.** Open a third Dataset for a new date. Merge this Dataset with the first two Datasets along the date dimension. There are a few ways to do this, but the easiest way is to create a new Dataset using the `concat` command and list all three date files in the brackets.

In [ ]:
# open a new Dataset


In [ ]:
# merge the three date Datasets together


**4.** Find the daily mean NO$_{2}$ concentration from the 3 merged dates. Plot a line plot for NO$_{2}$ concentration versus date.

In [ ]:
# find the mean


In [ ]:
# plot the data


### Possible Solutions

In [ ]:
# 1a
subset = merge[dict(row=slice(None,200))]
subset.dims

In [ ]:
# 1b
subset = merge[dict(col=slice(100,200),layer=33)]
subset.dims

In [ ]:
# 1c
subset = merge[dict(row=3,col=slice(20,50),layer=0)].assign_coords(layer='surface')
subset

In [ ]:
# 2
var = merge['avKern']

In [ ]:
# 2a
var.mean(axis=2)

In [ ]:
# 2b
var.mean(axis=2).mean(axis=1)

In [ ]:
# 2c
var.mean(['row','col','layer'])

In [ ]:
# 3
year = '2020'
month_name = 'Jan'
month_num = '01'
day = '03'
path = 'DOMINO_NO2_'+month_name+year
file = '/WHIPS-NO2-12kmCONUS_'+month_num+'-'+day+'-'+year+'.nc'

data3 = xr.open_dataset(path+file)

In [ ]:
bigmerge = xr.concat([data,data2,data3],dim='date')
bigmerge.dims

In [ ]:
# 4
NO2 = bigmerge['tropVCD']
daily = NO2.mean(['row','col'])

In [ ]:
fig = plt.figure(figsize=(10,8))
daily.plot(c='k')
plt.xlabel('Date',fontsize=12)
plt.ylabel('NO$_{2}$ (molecules/cm$^{2}$)',fontsize=12)
plt.xticks(np.arange(3),pd.date_range('01-01-2020','01-03-2020'))
plt.title('Daily NO$_{2}$ Concentration',fontsize=18)
plt.show()